https://github.com/pysal/mgwr/pull/60

In [9]:
import sys
sys.path.append("C:/Users/msachde1/Downloads/Research/Development/mgwr")

In [10]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

from mgwr.gwr import GWR
from spglm.family import Gaussian, Binomial, Poisson
from mgwr.gwr import MGWR
from mgwr.sel_bw import Sel_BW
import multiprocessing as mp
pool = mp.Pool()
from scipy import linalg
import numpy.linalg as la
from scipy import sparse as sp
from scipy.sparse import linalg as spla
from spreg.utils import spdot, spmultiply
from scipy import special
import libpysal as ps
import seaborn as sns
import matplotlib.pyplot as plt
from copy import deepcopy
import copy
from collections import namedtuple
import spglm

#### Clearwater data - downloaded from link: https://sgsup.asu.edu/sparc/multiscale-gwr

In [11]:
data_p = pd.read_csv("C:/Users/msachde1/Downloads/logistic_mgwr_data/landslides.csv") 

In [12]:
data_p.head()

,UserID,X,Y,Elev,Slope,SinAspct,CosAspct,AbsSouth,Landslid,DistStrm
0,1,616168.5625,5201076.5,1450.475,27.44172,0.409126,-0.912478,24.1499,1,8.506
1,2,624923.8125,5201008.5,1567.476,21.88343,-0.919245,-0.393685,66.8160,1,15.561
2,3,615672.0000,5199187.5,1515.065,38.81030,-0.535024,-0.844837,32.3455,1,41.238
3,4,615209.3125,5199112.0,1459.827,26.71631,-0.828548,-0.559918,55.9499,1,17.539
4,5,616354.6875,5198945.5,1379.442,27.55271,-0.872281,-0.489005,60.7248,1,35.023


### GWR Binomial model with independent variable, x = slope

In [13]:
coords = list(zip(data_p['X'],data_p['Y']))
y = np.array(data_p['Landslid']).reshape((-1,1)) 
elev = np.array(data_p['Elev']).reshape((-1,1))
slope = np.array(data_p['Slope']).reshape((-1,1))
SinAspct = np.array(data_p['SinAspct']).reshape(-1,1)
CosAspct = np.array(data_p['CosAspct']).reshape(-1,1)
X = np.hstack([elev,slope,SinAspct,CosAspct])
x = CosAspct

X_std = (X-X.mean(axis=0))/X.std(axis=0)
x_std = (x-x.mean(axis=0))/x.std(axis=0)
y_std = (y-y.mean(axis=0))/y.std(axis=0)

In [18]:
bw=Sel_BW(coords,y,x_std,family=Binomial(),constant=True).search()
gwr_mod=GWR(coords,y,x_std,bw=bw,family=Binomial(),constant=True).fit()
bw

70.0

##### Running the function with family = Binomial()

In [19]:
selector = Sel_BW(coords,y,x_std,family=Binomial(),multi=True,constant=True)
selector.search(verbose=True)

Current iteration: 1 ,SOC: 0.0329294
Bandwidths: 43.0, 108.0
Current iteration: 2 ,SOC: 0.0102171
Bandwidths: 43.0, 121.0
Current iteration: 3 ,SOC: 0.003753
Bandwidths: 43.0, 121.0
Current iteration: 4 ,SOC: 0.000857
Bandwidths: 43.0, 121.0
Current iteration: 5 ,SOC: 0.0002682
Bandwidths: 43.0, 121.0
Current iteration: 6 ,SOC: 0.0001019
Bandwidths: 43.0, 121.0
Current iteration: 7 ,SOC: 2.8e-05
Bandwidths: 43.0, 121.0
Current iteration: 8 ,SOC: 8.9e-06
Bandwidths: 43.0, 121.0


array([ 43., 121.])

In [20]:
mgwr_mod = MGWR(coords, y,x_std,selector,family=Binomial(),constant=True).fit()

In [21]:
mgwr_mod.bic

259.4259231912246

In [22]:
gwr_mod.bic

367.74360890575866

In [27]:
mgwr_mod.predy-gwr_mod.predy

array([[ 5.43587477e-02],
       [ 3.50984892e-02],
       [ 6.97002365e-02],
       [ 9.92455319e-02],
       [ 8.21845716e-02],
       [ 7.23566189e-02],
       [ 4.97091452e-02],
       [ 9.61095886e-02],
       [ 1.29691855e-01],
       [ 9.15624050e-02],
       [ 7.65380602e-02],
       [ 4.77719579e-02],
       [ 4.28259444e-02],
       [ 1.06149574e-01],
       [-5.05901412e-02],
       [-6.80163124e-02],
       [ 1.74201024e-01],
       [ 9.52572969e-02],
       [ 1.71968519e-01],
       [ 6.25317962e-02],
       [ 1.63916418e-01],
       [ 1.03138957e-01],
       [ 1.74192567e-01],
       [ 1.35120377e-01],
       [ 9.66095114e-02],
       [ 9.52747981e-02],
       [-1.19955166e-01],
       [ 1.22896447e-01],
       [ 1.28772698e-01],
       [ 4.28704831e-02],
       [ 1.61693089e-01],
       [-1.83347394e-02],
       [-2.42456015e-02],
       [-1.76216884e-02],
       [-1.27793892e-02],
       [ 1.01490246e-01],
       [ 1.09883229e-01],
       [ 1.12773653e-01],
       [ 1.1

#### Multiple bandwidth check

In [28]:
bw=Sel_BW(coords,y,X_std,family=Binomial(),constant=True).search()
gwr_mod=GWR(coords,y,X_std,bw=bw,family=Binomial(),constant=True).fit()
bw

121.0

### Checking parameters

In [30]:
selector = Sel_BW(coords,y,X_std,family=Binomial(),multi=True,constant=True)
selector.search(verbose=True)

Current iteration: 1 ,SOC: 0.061895
Bandwidths: 43.0, 56.0, 190.0, 120.0, 184.0
Current iteration: 2 ,SOC: 0.0192169
Bandwidths: 43.0, 46.0, 203.0, 120.0, 184.0
Current iteration: 3 ,SOC: 0.0071117
Bandwidths: 43.0, 46.0, 199.0, 120.0, 184.0
Current iteration: 4 ,SOC: 0.0008658
Bandwidths: 43.0, 46.0, 199.0, 120.0, 184.0
Current iteration: 5 ,SOC: 0.0002242
Bandwidths: 43.0, 46.0, 199.0, 120.0, 184.0
Current iteration: 6 ,SOC: 7.41e-05
Bandwidths: 43.0, 46.0, 199.0, 120.0, 184.0
Current iteration: 7 ,SOC: 2.97e-05
Bandwidths: 43.0, 46.0, 199.0, 120.0, 184.0
Current iteration: 8 ,SOC: 1.26e-05
Bandwidths: 43.0, 46.0, 199.0, 120.0, 184.0
Current iteration: 9 ,SOC: 5.6e-06
Bandwidths: 43.0, 46.0, 199.0, 120.0, 184.0


array([ 43.,  46., 199., 120., 184.])

In [31]:
mgwr_mod = MGWR(coords, y,X_std,selector,family=Binomial(),constant=True).fit()

In [33]:
gwr_mod.aicc, mgwr_mod.aicc

(264.9819711678866, 236.74179310353603)

#### Global model check

In [34]:
selector=Sel_BW(coords,y,X_std,multi=True,family=Binomial(),constant=True)
selector.search(verbose=True,multi_bw_min=[239,239,239,239,239], multi_bw_max=[239,239,239,239,239])

Current iteration: 1 ,SOC: 0.1355639
Bandwidths: 239.0, 239.0, 239.0, 239.0, 239.0
Current iteration: 2 ,SOC: 0.0355616
Bandwidths: 239.0, 239.0, 239.0, 239.0, 239.0
Current iteration: 3 ,SOC: 0.0077361
Bandwidths: 239.0, 239.0, 239.0, 239.0, 239.0
Current iteration: 4 ,SOC: 0.0005525
Bandwidths: 239.0, 239.0, 239.0, 239.0, 239.0
Current iteration: 5 ,SOC: 8.1e-05
Bandwidths: 239.0, 239.0, 239.0, 239.0, 239.0
Current iteration: 6 ,SOC: 1.77e-05
Bandwidths: 239.0, 239.0, 239.0, 239.0, 239.0
Current iteration: 7 ,SOC: 4.4e-06
Bandwidths: 239.0, 239.0, 239.0, 239.0, 239.0


array([239., 239., 239., 239., 239.])

In [35]:
mgwr_mod = MGWR(coords, y,X_std,selector,family=Binomial(),constant=True).fit()

In [36]:
gwr_mod.summary()

Model type                                                         Binomial
Number of observations:                                                 239
Number of covariates:                                                     5

Global Regression Results
---------------------------------------------------------------------------
Deviance:                                                           266.246
Log-likelihood:                                                    -133.123
AIC:                                                                276.246
AICc:                                                               276.504
BIC:                                                              -1015.246
Percent deviance explained:                                           0.182
Adj. percent deviance explained:                                      0.168

Variable                              Est.         SE  t(Est/SE)    p-value
------------------------------- ---------- ---------- ------

In [37]:
np.mean(mgwr_mod.params,axis=0)

array([ 0.57247827, -1.01119236,  1.07071867,  0.20218987, -0.61098986])

In [38]:
mgwr_mod.bic

292.95192410952586

In [39]:
gwr_mod.bic

340.5982180538755